In [4]:
import numpy as np
import time

In [5]:
CONFIG_LINES = 7

def read_case(fname, verbose=True):
    if verbose:
        print(f'Reading {fname}...')
    
    metadata = {}
    nodes_coords = []
    nodes_demand = []

    with open(fname, 'r') as f:
        for i in range(CONFIG_LINES):
            cfg = list(map(lambda x: x.strip(), f.readline().split(':', maxsplit=1)))
            if len(cfg) > 1:
                try:
                    metadata[cfg[0]] = int(cfg[1])
                except:
                    metadata[cfg[0]] = cfg[1]
            if verbose:
                print(" = ".join(cfg))
        
        metadata['NUM_TRUCKS'] = int(metadata['NAME'].split('-')[-1][1:])

        for i in range(metadata['DIMENSION']):
            cline = list(map(lambda x: int(x.strip()), f.readline().split()))
            assert len(cline) == 3

            nodes_coords.append(tuple(cline[1:]))

        if verbose:
            print(f.readline())

        for i in range(metadata['DIMENSION']):
            cline = list(map(lambda x: int(x.strip()), f.readline().split()))
            assert len(cline) == 2

            nodes_demand.append(cline[-1]) 
    
    return metadata, nodes_coords, nodes_demand

In [6]:
def dist(a, b):
    return np.sqrt(((a[0] - b[0])**2) + ((a[1] - b[1])**2))

def transfer_proba(i, j, alpha, beta, tau, nu, possible):
    num = (tau[i, j] ** alpha) * (nu[i, j] ** beta)
    denom = np.sum([(tau[i, z] ** alpha) * (nu[i, z] ** beta) for z in possible])

    return num / denom

def path_cost(path, coords):
    cost = 0
    for i in range(1, len(path)):
        cost += dist(coords[path[i - 1]], coords[path[i]])

    return cost

In [7]:
num_agens_per_epoch = 5
num_epoch = 10
alpha = 5
beta = 2.5
p = 0.1
Q = 64
tau_0 = 0.01

MAX_FIX_SOLUTION = 9999999 # не менять

dcfg, coords, demand = read_case('vrp_instances/A/A-n32-k5.vrp')

dim = dcfg['DIMENSION']
num_trucks = dcfg['NUM_TRUCKS']
cap = dcfg['CAPACITY']

tau = np.ones((dim, dim)) * tau_0
nu = np.zeros((dim, dim))

for i in range(dim):
    for j in range(i + 1, dim):
        d = dist(coords[i], coords[j])
        nu[i, j] = 1 / (d + 0.001) 

nu += nu.T

best_path = None
best_path_cost = None

start = time.perf_counter()

for epoch in range(num_epoch):
    print('--- epoch', epoch, '---')

    for _ in range(num_agens_per_epoch):
        print('agent', _)
        trying_fix = 0
        while True:
            possible = set(range(1, dim))
            path = [0]
            curr_demand = demand.copy()

            for i in range(num_trucks):
                curr_cap = cap
                curr_position = 0
                if 0 in possible:
                    possible.remove(0)
                while len(possible) > 0:
                    proba = [0] * dim

                    for pos in possible:
                        proba[pos] = transfer_proba(curr_position, pos, alpha, beta, tau, nu, possible)

                    new_pos = np.random.choice(dim, replace=False, p=proba)
                    if new_pos != 0:
                        possible.add(0)

                    if new_pos == 0:
                        path.append(0)
                        i += 1
                        break

                    decrease = min(curr_cap, demand[new_pos])
                    curr_demand[new_pos] -= decrease
                    curr_cap -= decrease
                    path.append(new_pos)
                    possible.remove(new_pos)
                    curr_position = new_pos

                    if curr_cap <= 0:
                        path.append(0)
                        break

            path.append(0)

            if np.sum(curr_demand) == 0 or (trying_fix == MAX_FIX_SOLUTION and epoch != num_epoch - 1):
                break
            else:
                trying_fix += 1

        
        pc = path_cost(path, coords)
        print('cost: ', pc)

        if best_path_cost == None or pc < best_path_cost:
            best_path_cost = pc
            best_path = path

        used_edges = []

        for i in range(1, len(path)):
            used_edges.append((path[i - 1], path[i]))

        if used_edges[-1] == (0, 0):
            used_edges = used_edges[:-1]

        assert len(used_edges) == len(set(used_edges))

        used_edges = set(used_edges)

        for i in range(dim):
            for j in range(dim):
                dt = 0
                if (i, j) in used_edges:
                    dt = tau_0 * nu[i, j]
                
                tau[i, j] = (1 - p) * tau[i, j] + p * dt

    used_edges = []
    path = best_path

    for i in range(1, len(path)):
        used_edges.append((path[i - 1], path[i]))

    if used_edges[-1] == (0, 0):
        used_edges = used_edges[:-1]

    assert len(used_edges) == len(set(used_edges))

    used_edges = set(used_edges)

    for edge in used_edges:
        dt = Q / best_path_cost

        tau[edge] = (1 - p) * tau[edge] + p * dt

end = time.perf_counter()
print()
print('best value:', best_path_cost)
print()
print('work time:', end - start, 's')


lastzero = 1

if best_path[-1] == 0 and best_path[-2] == 0:
    best_path = best_path[:-1]

route_num = 1

with open('ans6/' + dcfg['NAME'] + '.sol', 'w') as f:
    while lastzero < len(best_path):
        curr = best_path.index(0, lastzero)
        f.write(f'Route #{route_num}: ' + ' '.join(list(map(lambda x: str(x + 1), best_path[lastzero - 1:curr]))) + '\n')
        route_num += 1
        lastzero = curr + 1

    f.write(f'cost {best_path_cost}')

Reading vrp_instances/A/A-n32-k5.vrp...
NAME = A-n32-k5
COMMENT = (Augerat et al, No of trucks: 5, Optimal value: 784)
TYPE = CVRP
DIMENSION = 32
EDGE_WEIGHT_TYPE = EUC_2D
CAPACITY = 100
NODE_COORD_SECTION
DEMAND_SECTION 

--- epoch 0 ---
agent 0
cost:  1093.5869942412035
agent 1
cost:  1166.8337680995471
agent 2
cost:  1083.9606741645682
agent 3
cost:  1241.328523827344
agent 4
cost:  987.9908950977483
--- epoch 1 ---
agent 0
cost:  1010.5693592513538
agent 1
cost:  1004.2933891232179
agent 2
cost:  957.9032545463702
agent 3
cost:  1008.4281388661501
agent 4
cost:  1007.4143597971479
--- epoch 2 ---
agent 0
cost:  957.9032545463705
agent 1
cost:  959.4902901470141
agent 2
cost:  957.9032545463702
agent 3
cost:  957.9032545463702
agent 4
cost:  957.9032545463702
--- epoch 3 ---
agent 0
cost:  957.9032545463702
agent 1
cost:  957.9032545463702
agent 2
cost:  957.9032545463702
agent 3
cost:  957.9032545463702
agent 4
cost:  957.9032545463702
--- epoch 4 ---
agent 0
cost:  957.90325454637

In [ ]:
num_agens_per_epoch = 5
num_epoch = 10
alpha = 5
beta = 2.5
p = 0.1
Q = 64
tau_0 = 0.01

MAX_FIX_SOLUTION = 9999999 # не менять

dcfg, coords, demand = read_case('vrp_instances/A/A-n39-k6.vrp')

dim = dcfg['DIMENSION']
num_trucks = dcfg['NUM_TRUCKS']
cap = dcfg['CAPACITY']

tau = np.ones((dim, dim)) * tau_0
nu = np.zeros((dim, dim))

for i in range(dim):
    for j in range(i + 1, dim):
        d = dist(coords[i], coords[j])
        nu[i, j] = 1 / (d + 0.001) 

nu += nu.T

best_path = None
best_path_cost = None

start = time.perf_counter()

for epoch in range(num_epoch):
    print('--- epoch', epoch, '---')

    for _ in range(num_agens_per_epoch):
        print('agent', _)
        trying_fix = 0
        while True:
            possible = set(range(1, dim))
            path = [0]
            curr_demand = demand.copy()

            for i in range(num_trucks):
                curr_cap = cap
                curr_position = 0
                if 0 in possible:
                    possible.remove(0)
                while len(possible) > 0:
                    proba = [0] * dim

                    for pos in possible:
                        proba[pos] = transfer_proba(curr_position, pos, alpha, beta, tau, nu, possible)

                    new_pos = np.random.choice(dim, replace=False, p=proba)
                    if new_pos != 0:
                        possible.add(0)

                    if new_pos == 0:
                        path.append(0)
                        i += 1
                        break

                    decrease = min(curr_cap, demand[new_pos])
                    curr_demand[new_pos] -= decrease
                    curr_cap -= decrease
                    path.append(new_pos)
                    possible.remove(new_pos)
                    curr_position = new_pos

                    if curr_cap <= 0:
                        path.append(0)
                        break

            path.append(0)

            if np.sum(curr_demand) == 0 or (trying_fix == MAX_FIX_SOLUTION and epoch != num_epoch - 1):
                break
            else:
                trying_fix += 1

        
        pc = path_cost(path, coords)
        print('cost: ', pc)

        if best_path_cost == None or pc < best_path_cost:
            best_path_cost = pc
            best_path = path

        used_edges = []

        for i in range(1, len(path)):
            used_edges.append((path[i - 1], path[i]))

        if used_edges[-1] == (0, 0):
            used_edges = used_edges[:-1]

        assert len(used_edges) == len(set(used_edges))

        used_edges = set(used_edges)

        for i in range(dim):
            for j in range(dim):
                dt = 0
                if (i, j) in used_edges:
                    dt = tau_0 * nu[i, j]
                
                tau[i, j] = (1 - p) * tau[i, j] + p * dt

    used_edges = []
    path = best_path

    for i in range(1, len(path)):
        used_edges.append((path[i - 1], path[i]))

    if used_edges[-1] == (0, 0):
        used_edges = used_edges[:-1]

    assert len(used_edges) == len(set(used_edges))

    used_edges = set(used_edges)

    for edge in used_edges:
        dt = Q / best_path_cost

        tau[edge] = (1 - p) * tau[edge] + p * dt

end = time.perf_counter()
print()
print('best value:', best_path_cost)
print()
print('work time:', end - start, 's')


lastzero = 1

if best_path[-1] == 0 and best_path[-2] == 0:
    best_path = best_path[:-1]

route_num = 1

with open('ans6/' + dcfg['NAME'] + '.sol', 'w') as f:
    while lastzero < len(best_path):
        curr = best_path.index(0, lastzero)
        f.write(f'Route #{route_num}: ' + ' '.join(list(map(lambda x: str(x + 1), best_path[lastzero - 1:curr]))) + '\n')
        route_num += 1
        lastzero = curr + 1

    f.write(f'cost {best_path_cost}')

Reading vrp_instances/A/A-n39-k6.vrp...
NAME = A-n39-k6
COMMENT = (Augerat et al, No of trucks: 6, Optimal value: 831)
TYPE = CVRP
DIMENSION = 39
EDGE_WEIGHT_TYPE = EUC_2D
CAPACITY = 100
NODE_COORD_SECTION
DEMAND_SECTION 

--- epoch 0 ---
agent 0
cost:  1223.5024247898223
agent 1
cost:  1222.1879182410148
agent 2
cost:  1168.3317429802562
agent 3


In [1]:
lastzero = 1

if best_path[-1] == 0 and best_path[-2] == 0:
    best_path = best_path[:-1]

route_num = 1

with open('ans6/' + dcfg['NAME'] + '.sol', 'w') as f:
    while lastzero < len(best_path):
        curr = best_path.index(0, lastzero)
        f.write(f'Route #{route_num}: ' + ' '.join(list(map(lambda x: str(x + 1), best_path[lastzero - 1:curr]))) + '\n')
        route_num += 1
        lastzero = curr + 1

    f.write(f'cost {best_path_cost}')

NameError: name 'best_path' is not defined